# **Laboratorio 11: LLM y Agentes Autónomos 🤖**

MDS7202: Laboratorio de Programación Científica para Ciencia de Datos

### **Cuerpo Docente:**

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### **Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados**

- Nombre de alumno 1: Cristopher Urbina H.
- Nombre de alumno 2: Joaquín Zamora O.



### **Link de repositorio de GitHub:** [Repositorio](https://github.com/CrisU8/MDS7202-Primavera2024)


## **Temas a tratar**

- Reinforcement Learning
- Large Language Models

## **Reglas:**

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.

### **Objetivos principales del laboratorio**

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

## **1. Reinforcement Learning (2.0 puntos)**

En esta sección van a usar métodos de RL para resolver dos problemas interesantes: `Blackjack` y `LunarLander`.

In [ ]:
!pip install -qqq gymnasium stable_baselines3
!pip install -qqq swig
!pip install -qqq gymnasium[box2d]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


### **1.1 Blackjack (1.0 puntos)**

<p align="center">
  <img src="https://www.recreoviral.com/wp-content/uploads/2016/08/s3.amazonaws.com-Math.gif"
" width="400">
</p>

La idea de esta subsección es que puedan implementar métodos de RL y así generar una estrategia para jugar el clásico juego Blackjack y de paso puedan ~~hacerse millonarios~~ aprender a resolver problemas mediante RL.

Comencemos primero preparando el ambiente. El siguiente bloque de código transforma las observaciones del ambiente a `np.array`:


In [ ]:
import gymnasium as gym
from gymnasium.spaces import MultiDiscrete
import numpy as np

class FlattenObservation(gym.ObservationWrapper):
    def __init__(self, env):
        super(FlattenObservation, self).__init__(env)
        self.observation_space = MultiDiscrete(np.array([32, 11, 2]))

    def observation(self, observation):
        return np.array(observation).flatten()

# Create and wrap the environment
env = gym.make("Blackjack-v1")
env = FlattenObservation(env)

#### **1.1.1 Descripción de MDP (0.2 puntos)**

Entregue una breve descripción sobre el ambiente [Blackjack](https://gymnasium.farama.org/environments/toy_text/blackjack/) y su formulación en MDP, distinguiendo de forma clara y concisa los estados, acciones y recompensas.


### 1. **Estados (Observaciones):**
La información que el agente recibe del entorno:
- **Suma del jugador:** \(4 <= 21\).
- **Carta visible del dealer:** \(1 <= 10\).
- **As usable:** \(0\) o \(1\).

El espacio de estados se representa como un vector:
\[
(\text{Player's sum}, \text{Dealer's card}, \text{Usable ace})
\]

### 2. **Acciones:**
Conjunto de decisiones posibles:
- \(0\): **Stick** (no pedir más cartas).
- \(1\): **Hit** (pedir una carta adicional).

El espacio de acciones es discreto con dos opciones (\(\{0, 1\}\)).

### 3. **Recompensas:**
Feedback otorgado al agente por ejecutar una acción:
- \(+1.5\): Ganar con un blackjack natural (si se permite esta regla).
- \(+1\): Ganar.
- \(0\): Empatar.
- \(-1\): Perder (por bust o puntuación menor que la del dealer).

### Dinámica del MDP
- **Transiciones**:
  - Las acciones afectan el estado del jugador (suma de cartas y condición de bust).
  - El dealer actúa automáticamente siguiendo reglas fijas cuando el jugador elige "stick".
- **Probabilidades**:
  - Las cartas son seleccionadas al azar (baraja infinita).








#### **1.1.2 Generando un Baseline (0.2 puntos)**

Simule un escenario en donde se escojan acciones aleatorias. Repita esta simulación 5000 veces y reporte el promedio y desviación de las recompensas. ¿Cómo calificaría el performance de esta política? ¿Cómo podría interpretar las recompensas obtenidas?

In [ ]:
# Simulate 5000 games with random actions
n_simulations = 5000
rewards = []

for _ in range(n_simulations):
    state = env.reset()
    done = False
    total_reward = 0
    while not done:
        action = env.action_space.sample()  # Take a random action
        _, reward, done, _, _ = env.step(action)
        total_reward += reward  # Accumulate reward for the episode
    rewards.append(total_reward)

# Calculate mean and standard deviation of rewards
mean_reward = np.mean(rewards)
std_reward = np.std(rewards)

print(f"Mean Reward: {mean_reward}")
print(f"Standard Deviation of Rewards: {std_reward}")

Mean Reward: -0.3848
Standard Deviation of Rewards: 0.8977354621490676


#### **1.1.3 Entrenamiento de modelo (0.2 puntos)**

A partir del siguiente [enlace](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html), escoja un modelo de `stable_baselines3` y entrenelo para resolver el ambiente `Blackjack`.

In [ ]:
import gymnasium as gym
from gymnasium.spaces import Box
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

# Wrapper para aplanar el espacio de observación
class FlattenObservation(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        # Cambiar el espacio de observación a Box
        self.observation_space = Box(low=0, high=32, shape=(3,), dtype=np.float32)

    def observation(self, obs):
        # Aplanar las observaciones a un vector
        return np.array(obs, dtype=np.float32)

# Crear el entorno con el wrapper aplicado
def make_blackjack_env():
    env = gym.make("Blackjack-v1", natural=True)
    return FlattenObservation(env)

# Crear el entorno vectorizado
env = make_vec_env(make_blackjack_env, n_envs=1)

# Inicializar el modelo PPO
model = PPO("MlpPolicy", env, verbose=1)

# Entrenar el modelo PPO
model.learn(total_timesteps=50000)  # Ajusta según los recursos disponibles

# Guardar el modelo entrenado
model.save("ppo_blackjack")
print("Modelo entrenado y guardado como 'ppo_blackjack'.")

Using cpu device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.48     |
|    ep_rew_mean     | -0.3     |
| time/              |          |
|    fps             | 841      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.26        |
|    ep_rew_mean          | -0.48       |
| time/                   |             |
|    fps                  | 610         |
|    iterations           | 2           |
|    time_elapsed         | 6           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.019496009 |
|    clip_fraction        | 0.361       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.674      |
|    explained_variance   | -0.00731    |
|    learning

#### **1.1.4 Evaluación de modelo (0.2 puntos)**

Repita el ejercicio 1.1.2 pero utilizando el modelo entrenado. ¿Cómo es el performance de su agente? ¿Es mejor o peor que el escenario baseline?

In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy

# Asegurarse de que eval_env está definido
eval_env = make_vec_env(make_blackjack_env, n_envs=1)

# Evaluar el modelo entrenado
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=5000, deterministic=True)

print(f"Recompensa Media del Modelo Entrenado: {mean_reward}")
print(f"Desviación Estándar del Modelo Entrenado: {std_reward}")

Recompensa Media del Modelo Entrenado: -0.0514
Desviación Estándar del Modelo Entrenado: 0.9551743505768987


#### **1.1.5 Estudio de acciones (0.2 puntos)**

Genere una función que reciba un estado y retorne la accion del agente. Luego, use esta función para entregar la acción escogida frente a los siguientes escenarios:

- Suma de cartas del agente es 6, dealer muestra un 7, agente no tiene tiene un as
- Suma de cartas del agente es 19, dealer muestra un 3, agente tiene tiene un as

¿Son coherentes sus acciones con las reglas del juego?

Hint: ¿A que clase de python pertenecen los estados? Pruebe a usar el método `.reset` para saberlo.

In [ ]:
import numpy as np

def get_agent_action(state):
    # Convertir el estado en un array de tipo float32
    state_array = np.array(state, dtype=np.float32)

    # Asegurarnos de que el estado tenga la forma correcta (1, 3)
    state_array = state_array.reshape(1, -1)

    # Obtener la acción del modelo entrenado
    action, _ = model.predict(state_array, deterministic=True)

    # Extraer la acción (0: plantarse, 1: pedir carta)
    return action[0]

# Escenario 1
state1 = (6, 7, False)
action1 = get_agent_action(state1)
print(f"Acción del agente en el Escenario 1: {'Pedir carta' if action1 == 1 else 'Plantarse'}")

# Escenario 2
state2 = (19, 3, True)
action2 = get_agent_action(state2)
print(f"Acción del agente en el Escenario 2: {'Pedir carta' if action2 == 1 else 'Plantarse'}")


Acción del agente en el Escenario 1: Pedir carta
Acción del agente en el Escenario 2: Plantarse


In [ ]:
import gymnasium as gym
env = gym.make("LunarLander-v3", render_mode = "rgb_array", continuous = True) # notar el parámetro continuous = True

/usr/local/lib/python3.10/dist-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.cloud')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-pa

Noten que se especifica el parámetro `continuous = True`. ¿Que implicancias tiene esto sobre el ambiente?

Además, se le facilita la función `export_gif` para el ejercicio 2.2.4:

In [ ]:
import imageio
import numpy as np

def export_gif(model, n = 5):
  '''
  función que exporta a gif el comportamiento del agente en n episodios
  '''
  images = []
  for episode in range(n):
    obs = model.env.reset()
    img = model.env.render()
    done = False
    while not done:
      images.append(img)
      action, _ = model.predict(obs)
      obs, reward, done, info = model.env.step(action)
      img = model.env.render(mode="rgb_array")

  imageio.mimsave("agent_performance.gif", [np.array(img) for i, img in enumerate(images) if i%2 == 0], fps=29)

#### **1.2.1 Descripción de MDP (0.2 puntos)**

Entregue una breve descripción sobre el ambiente [LunarLander](https://gymnasium.farama.org/environments/box2d/lunar_lander/) y su formulación en MDP, distinguiendo de forma clara y concisa los estados, acciones y recompensas. ¿Como se distinguen las acciones de este ambiente en comparación a `Blackjack`?

Nota: recuerde que se especificó el parámetro `continuous = True`

`escriba su respuesta acá`

#### **1.2.2 Generando un Baseline (0.2 puntos)**

Simule un escenario en donde se escojan acciones aleatorias. Repita esta simulación 10 veces y reporte el promedio y desviación de las recompensas. ¿Cómo calificaría el performance de esta política?

In [ ]:
import gymnasium as gym
import numpy as np

# Crear el entorno LunarLander con acciones continuas
env = gym.make("LunarLander-v3", continuous=True)

n_episodes = 10
total_rewards = []

for episode in range(n_episodes):
    obs, info = env.reset()
    done = False
    episode_reward = 0
    while not done:
        # Seleccionar una acción aleatoria
        action = env.action_space.sample()
        obs, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        episode_reward += reward
    total_rewards.append(episode_reward)
    print(f"Recompensa del Episodio {episode + 1}: {episode_reward:.2f}")

env.close()

# Calcular estadísticas
mean_reward = np.mean(total_rewards)
std_reward = np.std(total_rewards)

print(f"\nRecompensa Media: {mean_reward:.2f}")
print(f"Desviación Estándar: {std_reward:.2f}")

Recompensa del Episodio 1: -276.31
Recompensa del Episodio 2: -48.98
Recompensa del Episodio 3: -161.62
Recompensa del Episodio 4: -458.87
Recompensa del Episodio 5: -109.33
Recompensa del Episodio 6: -22.06
Recompensa del Episodio 7: -232.30
Recompensa del Episodio 8: -107.85
Recompensa del Episodio 9: -205.06
Recompensa del Episodio 10: -66.31

Recompensa Media: -168.87
Desviación Estándar: 124.40


#### **1.2.3 Entrenamiento de modelo (0.2 puntos)**

A partir del siguiente [enlace](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html), escoja un modelo de `stable_baselines3` y entrenelo para resolver el ambiente `LunarLander` **usando 10000 timesteps de entrenamiento**.

In [ ]:
import gymnasium as gym
from stable_baselines3 import SAC

# Crear el entorno LunarLander con acciones continuas
env = gym.make("LunarLander-v3", continuous=True)

# Crear el modelo SAC
model = SAC("MlpPolicy", env, verbose=1)

# Entrenar el modelo durante 10,000 timesteps
model.learn(total_timesteps=10000)

# Guardar el modelo entrenado
model.save("sac_lunarlander")
print("Modelo entrenado y guardado como 'sac_lunarlander'.")


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 113      |
|    ep_rew_mean     | -303     |
| time/              |          |
|    episodes        | 4        |
|    fps             | 40       |
|    time_elapsed    | 11       |
|    total_timesteps | 453      |
| train/             |          |
|    actor_loss      | 1.85     |
|    critic_loss     | 52.2     |
|    ent_coef        | 0.903    |
|    ent_coef_loss   | -0.293   |
|    learning_rate   | 0.0003   |
|    n_updates       | 352      |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 127      |
|    ep_rew_mean     | -268     |
| time/              |          |
|    episodes        | 8        |
|    fps             | 36       |
|    time_elapsed    | 27       |
|    total_timesteps | 1018     |
| train/             |

#### **1.2.4 Evaluación de modelo (0.2 puntos)**

Repita el ejercicio 1.2.2 pero utilizando el modelo entrenado. ¿Cómo es el performance de su agente? ¿Es mejor o peor que el escenario baseline?

In [ ]:
import numpy as np

# Cargar el modelo entrenado (si no está ya en memoria)
# model = SAC.load("sac_lunarlander")

# Crear un entorno de evaluación
eval_env = gym.make("LunarLander-v3", continuous=True)

n_episodes = 10
total_rewards = []

for episode in range(n_episodes):
    obs, info = eval_env.reset()
    done = False
    episode_reward = 0
    while not done:
        # Predecir la acción usando el modelo entrenado
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, terminated, truncated, info = eval_env.step(action)
        done = terminated or truncated
        episode_reward += reward
    total_rewards.append(episode_reward)
    print(f"Recompensa del Episodio {episode + 1}: {episode_reward:.2f}")

eval_env.close()

# Calcular estadísticas
mean_reward = np.mean(total_rewards)
std_reward = np.std(total_rewards)

print(f"\nRecompensa Media del Modelo Entrenado: {mean_reward:.2f}")
print(f"Desviación Estándar: {std_reward:.2f}")


Recompensa del Episodio 1: -155.83
Recompensa del Episodio 2: -158.93
Recompensa del Episodio 3: -26.95
Recompensa del Episodio 4: -103.27
Recompensa del Episodio 5: -77.49
Recompensa del Episodio 6: -110.86
Recompensa del Episodio 7: -46.47
Recompensa del Episodio 8: -107.64
Recompensa del Episodio 9: -109.01
Recompensa del Episodio 10: -159.18

Recompensa Media del Modelo Entrenado: -105.56
Desviación Estándar: 43.31


#### **1.2.5 Optimización de modelo (0.2 puntos)**

Repita los ejercicios 1.2.3 y 1.2.4 hasta obtener un nivel de recompensas promedio mayor a 50. Para esto, puede cambiar manualmente parámetros como:
- `total_timesteps`
- `learning_rate`
- `batch_size`

Una vez optimizado el modelo, use la función `export_gif` para estudiar el comportamiento de su agente en la resolución del ambiente y comente sobre sus resultados.

Adjunte el gif generado en su entrega (mejor aún si además adjuntan el gif en el markdown).

In [ ]:
import gymnasium as gym
from stable_baselines3 import SAC

# Crear el entorno
env = gym.make("LunarLander-v3", continuous=True)

# Ajustar hiperparámetros
learning_rate = 3e-4
batch_size = 256
total_timesteps = 100000  # Incrementamos el número de timesteps

# Crear el modelo con los nuevos hiperparámetros
model = SAC(
    "MlpPolicy",
    env,
    learning_rate=learning_rate,
    batch_size=batch_size,
    verbose=1,
)

# Entrenar el modelo
model.learn(total_timesteps=total_timesteps)

# Guardar el modelo optimizado
model.save("sac_lunarlander_optimized")
print("Modelo optimizado y guardado como 'sac_lunarlander_optimized'.")


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 107      |
|    ep_rew_mean     | -173     |
| time/              |          |
|    episodes        | 4        |
|    fps             | 43       |
|    time_elapsed    | 9        |
|    total_timesteps | 428      |
| train/             |          |
|    actor_loss      | -0.582   |
|    critic_loss     | 43.2     |
|    ent_coef        | 0.91     |
|    ent_coef_loss   | -0.276   |
|    learning_rate   | 0.0003   |
|    n_updates       | 327      |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 146      |
|    ep_rew_mean     | -182     |
| time/              |          |
|    episodes        | 8        |
|    fps             | 36       |
|    time_elapsed    | 32       |
|    total_timesteps | 1172     |
| train/             |

## **2. Large Language Models (4.0 puntos)**

En esta sección se enfocarán en habilitar un Chatbot que nos permita responder preguntas útiles a través de LLMs.

### **2.0 Configuración Inicial**

<p align="center">
  <img src="https://media1.tenor.com/m/uqAs9atZH58AAAAd/config-config-issue.gif"
" width="400">
</p>

Como siempre, cargamos todas nuestras API KEY al entorno:

In [ ]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

if "TAVILY_API_KEY" not in os.environ:
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key: ")

### **2.1 Retrieval Augmented Generation (1.5 puntos)**

<p align="center">
  <img src="https://y.yarn.co/218aaa02-c47e-4ec9-b1c9-07792a06a88f_text.gif"
" width="400">
</p>

El objetivo de esta subsección es que habiliten un chatbot que pueda responder preguntas usando información contenida en documentos PDF a través de **Retrieval Augmented Generation.**

#### **2.1.1 Reunir Documentos (0 puntos)**

Reuna documentos PDF sobre los que hacer preguntas siguiendo las siguientes instrucciones:
  - 2 documentos .pdf como mínimo.
  - 50 páginas de contenido como mínimo entre todos los documentos.
  - Ideas para documentos: Documentos relacionados a temas académicos, laborales o de ocio. Aprovechen este ejercicio para construir algo útil y/o relevante para ustedes!
  - Deben ocupar documentos reales, no pueden utilizar los mismos de la clase.
  - Deben registrar sus documentos en la siguiente [planilla](https://docs.google.com/spreadsheets/d/1Hy1w_dOiG2UCHJ8muyxhdKPZEPrrL7BNHm6E90imIIM/edit?usp=sharing). **NO PUEDEN USAR LOS MISMOS DOCUMENTOS QUE OTRO GRUPO**
  - **Recuerden adjuntar los documentos en su entrega**.

In [ ]:
%pip install --upgrade --quiet PyPDF2

In [ ]:
import PyPDF2

doc_paths = [] # rellenar con los path a sus documentos

assert len(doc_paths) >= 2, "Deben adjuntar un mínimo de 2 documentos"

total_paginas = sum(len(PyPDF2.PdfReader(open(doc, "rb")).pages) for doc in doc_paths)
assert total_paginas >= 50, f"Páginas insuficientes: {total_paginas}"

#### **2.1.2 Vectorizar Documentos (0.2 puntos)**

Vectorice los documentos y almacene sus representaciones de manera acorde.

#### **2.1.3 Habilitar RAG (0.3 puntos)**

Habilite la solución RAG a través de una *chain* y guárdela en una variable.

#### **2.1.4 Verificación de respuestas (0.5 puntos)**

Genere un listado de 3 tuplas ("pregunta", "respuesta correcta") y analice la respuesta de su solución para cada una. ¿Su solución RAG entrega las respuestas que esperaba?

Ejemplo de tupla:
- Pregunta: ¿Quién es el presidente de Chile?
- Respuesta correcta: El presidente de Chile es Gabriel Boric

#### **2.1.5 Sensibilidad de Hiperparámetros (0.5 puntos)**

Extienda el análisis del punto 2.1.4 analizando cómo cambian las respuestas entregadas cambiando los siguientes hiperparámetros:
- `Tamaño del chunk`. (*¿Cómo repercute que los chunks sean mas grandes o chicos?*)
- `La cantidad de chunks recuperados`. (*¿Qué pasa si se devuelven muchos/pocos chunks?*)
- `El tipo de búsqueda`. (*¿Cómo afecta el tipo de búsqueda a las respuestas de mi RAG?*)

### **2.2 Agentes (1.0 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/rcqnN2aJCSEAAAAd/secret-agent-man.gif"
" width="400">
</p>

Similar a la sección anterior, en esta sección se busca habilitar **Agentes** para obtener información a través de tools y así responder la pregunta del usuario.

#### **2.2.1 Tool de Tavily (0.2 puntos)**

Generar una *tool* que pueda hacer consultas al motor de búsqueda **Tavily**.

#### **2.2.2 Tool de Wikipedia (0.2 puntos)**

Generar una *tool* que pueda hacer consultas a **Wikipedia**.

*Hint: Le puede ser de ayuda el siguiente [link](https://python.langchain.com/v0.1/docs/modules/tools/).*

#### **2.2.3 Crear Agente (0.3 puntos)**

Crear un agente que pueda responder preguntas preguntas usando las *tools* antes generadas. Asegúrese que su agente responda en español. Por último, guarde el agente en una variable.

#### **2.2.4 Verificación de respuestas (0.3 puntos)**

Pruebe el funcionamiento de su agente y asegúrese que el agente esté ocupando correctamente las tools disponibles. ¿En qué casos el agente debería ocupar la tool de Tavily? ¿En qué casos debería ocupar la tool de Wikipedia?

### **2.3 Multi Agente (1.5 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/r7QMJLxU4BoAAAAd/this-is-getting-out-of-hand-star-wars.gif"
" width="450">
</p>

El objetivo de esta subsección es encapsular las funcionalidades creadas en una solución multiagente con un **supervisor**.


#### **2.3.1 Generando Tools (0.5 puntos)**

Transforme la solución RAG de la sección 2.1 y el agente de la sección 2.2 a *tools* (una tool por cada uno).

#### **2.3.2 Agente Supervisor (0.5 puntos)**

Habilite un agente que tenga acceso a las tools del punto anterior y pueda responder preguntas relacionadas. Almacene este agente en una variable llamada supervisor.

#### **2.3.3 Verificación de respuestas (0.25 puntos)**

Pruebe el funcionamiento de su agente repitiendo las preguntas realizadas en las secciones 2.1.4 y 2.2.4 y comente sus resultados. ¿Cómo varían las respuestas bajo este enfoque?

#### **2.3.4 Análisis (0.25 puntos)**

¿Qué diferencias tiene este enfoque con la solución *Router* vista en clases? Nombre al menos una ventaja y desventaja.

`escriba su respuesta acá`

### **2.4 Memoria (Bonus +0.5 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/Gs95aiElrscAAAAd/memory-unlocked-ratatouille-critic.gif"
" width="400">
</p>

Una de las principales falencias de las soluciones que hemos visto hasta ahora es que nuestro chat no responde las interacciones anteriores, por ejemplo:

- Pregunta 1: "Hola! mi nombre es Sebastián"
  - Respuesta esperada: "Hola Sebastián! ..."
- Pregunta 2: "Cual es mi nombre?"
  - Respuesta actual: "Lo siento pero no conozco tu nombre :("
  - **Respuesta esperada: "Tu nombre es Sebastián"**

Para solucionar esto, se les solicita agregar un componente de **memoria** a la solución entregada en el punto 2.3.

**Nota: El Bonus es válido <u>sólo para la sección 2 de Large Language Models.</u>**

### **2.5 Despliegue (0 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/IytHqOp52EsAAAAd/you-get-a-deploy-deploy.gif"
" width="400">
</p>

Una vez tengan los puntos anteriores finalizados, toca la etapa de dar a conocer lo que hicimos! Para eso, vamos a desplegar nuestro modelo a través de `gradio`, una librería especializada en el levantamiento rápido de demos basadas en ML.

Primero instalamos la librería:

In [ ]:
%pip install --upgrade --quiet gradio

Luego sólo deben ejecutar el siguiente código e interactuar con la interfaz a través del notebook o del link generado:

In [ ]:
import gradio as gr
import time

def agent_response(message, history):
  '''
  Función para gradio, recibe mensaje e historial, devuelte la respuesta del chatbot.
  '''
  # get chatbot response
  response = ... # rellenar con la respuesta de su chat

  # assert
  assert type(response) == str, "output de route_question debe ser string"

  # "streaming" response
  for i in range(len(response)):
    time.sleep(0.015)
    yield response[: i+1]

gr.ChatInterface(
    agent_response,
    type="messages",
    title="Chatbot MDS7202", # Pueden cambiar esto si lo desean
    description="Hola! Soy un chatbot muy útil :)", # también la descripción
    theme="soft",
    ).launch(
        share=True, # pueden compartir el link a sus amig@s para que interactuen con su chat!
        debug = False,
        )